# Province Gender Available Twitter Data - Monthly Topics

In [1]:
import gzip
import json
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords

## Read Data from Gzip

In [2]:
with gzip.open("province_gender_available_metadata_added-220522_combined.txt.gz", "rb") as f:
    users = f.readlines()
    
for i in range(len(users)):
    users[i] = json.loads(users[i].decode("utf-8"))

## Create DataFrame

In [3]:
df = pd.DataFrame(users)

In [4]:
df.shape

(364570, 15)

In [5]:
df.head(3)

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,downloaded,pp
0,"adana, türkiye",nefes alıyorsak hala bir umut var demektir. fa...,ali,real107171,1158321552129187840,Mon Aug 05 10:19:13 +0000 2019,"[{'source': 'location', 'pcode': 1}]","[{'source': 'name', 'gender': 'male'}]","[126571977, 1974277500, 2784055768, 1097258683...","[721395301274238976, 1070013184344510464, 1510...","[{'twt_id_str': '1173723046072475649', 'twt_tx...",3,9,220318,1158321652691886080/VJc3tvbT_normal.jpg
1,"çanakkale, türkiye",emu-ı.t.,yahya,yahyahlvoglu,1702767367,Mon Aug 26 19:26:40 +0000 2013,"[{'source': 'location', 'pcode': 17}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[2390341533, 767691353144827906, 259925559, 34...","[2390341533, 957649186064949248, 1120358786097...","[{'ref_twt_id_str': '1421381248400994307', 're...",270,205,220318,1430022847607488516/KLhPDuBE_normal.jpg
2,"gaziantep, türkiye",galatasaray,can altay,canalta29003836,1161346708883816448,Tue Aug 13 18:40:06 +0000 2019,"[{'source': 'location', 'pcode': 27}]","[{'source': 'name', 'gender': 'male'}]","[1401495806142734337, 1245674500039102464, 983...","[1401495806142734337, 1290700623801876487, 124...","[{'twt_id_str': '1269646996362670081', 'twt_tx...",10,13,220318,1228998578703683585/Uq1zinZ-_normal.jpg


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364570 entries, 0 to 364569
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   location         364570 non-null  object
 1   description      364570 non-null  object
 2   name             364570 non-null  object
 3   screen_name      364570 non-null  object
 4   id_str           364570 non-null  object
 5   created_at       364570 non-null  object
 6   province_codes   364570 non-null  object
 7   genders          364570 non-null  object
 8   following        364570 non-null  object
 9   followers        364570 non-null  object
 10  tweets           364570 non-null  object
 11  followers_count  364570 non-null  object
 12  following_count  364570 non-null  object
 13  downloaded       364570 non-null  object
 14  pp               364570 non-null  object
dtypes: object(15)
memory usage: 41.7+ MB


## Filter for Favorites

In [7]:
# Filter "tweets" for favorite tweets
for i in range(len(df)):
    df.loc[i, "tweets"] = [tweet for tweet in df.loc[i, "tweets"] if tweet["type"] == "fav"]

In [8]:
fav_tweet_count = 0

for tweet_list in df["tweets"]:
    fav_tweet_count += len(tweet_list)
    
print(f"Favorite tweet count: {fav_tweet_count}")

Favorite tweet count: 11068724


In [9]:
fav_tweet_count_2021 = 0

for tweet_list in df["tweets"]:
    for tweet in tweet_list:
        if tweet["twt_date"][:2] == '21':
            fav_tweet_count_2021 += 1
            
print(f"Favorite tweet count for 2021: {fav_tweet_count_2021}")

Favorite tweet count for 2021: 2395175


In [10]:
fav_tweet_count_2022 = 0

for tweet_list in df["tweets"]:
    for tweet in tweet_list:
        if tweet["twt_date"][:2] == '22':
            fav_tweet_count_2022 += 1
            
print(f"Favorite tweet count for 2022: {fav_tweet_count_2022}")

Favorite tweet count for 2022: 3736420


## Find Monthly Topics

In [11]:
# Turkish Stop Words
trstop = [
    'a', 'acaba', 'altı', 'altmış', 'ama', 'ancak', 'arada', 'artık', 'asla', 'aslında', 'aslında', 'ayrıca', 'az', 'bana',
    'bazen', 'bazı', 'bazıları', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beş', 'bile', 'bilhassa', 'bin', 'bir',
    'biraz', 'birçoğu', 'birçok', 'biri', 'birisi', 'birkaç', 'birşey', 'biz', 'bizden', 'bize', 'bizi', 'bizim', 'böyle',
    'böylece', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'burada', 'bütün', 'çoğu',
    'çoğunu', 'çok', 'çünkü', 'da', 'daha', 'dahi', 'dan', 'de', 'defa', 'değil', 'diğer', 'diğeri', 'diğerleri', 'diye',
    'doksan', 'dokuz', 'dolayı', 'dolayısıyla', 'dört', 'e', 'edecek', 'eden', 'ederek', 'edilecek', 'ediliyor', 'edilmesi',
    'ediyor', 'eğer', 'elbette', 'elli', 'en', 'etmesi', 'etti', 'ettiği', 'ettiğini', 'fakat', 'falan', 'filan', 'gene',
    'gereği', 'gerek', 'gibi', 'göre', 'hala', 'halde', 'halen', 'hangi', 'hangisi', 'hani', 'hatta', 'hem', 'henüz', 'hep',
    'hepsi', 'her', 'herhangi', 'herkes', 'herkese', 'herkesi', 'herkesin', 'hiç', 'hiçbir', 'hiçbiri', 'i', 'ı', 'için',
    'içinde', 'iki', 'ile', 'ilgili', 'ise', 'işte', 'itibaren', 'itibariyle', 'kaç', 'kadar', 'karşın', 'kendi', 'kendilerine',
    'kendine', 'kendini', 'kendisi', 'kendisine', 'kendisini', 'kez', 'ki', 'kim', 'kime', 'kimi', 'kimin', 'kimisi', 'kimse',
    'kırk', 'madem', 'mi', 'mı', 'milyar', 'milyon', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nedenle', 'nerde', 'nerede', 'nereye',
    'neyse', 'niçin', 'nin', 'nın', 'niye', 'nun', 'nün', 'o', 'öbür', 'olan', 'olarak', 'oldu', 'olduğu', 'olduğunu',
    'olduklarını', 'olmadı', 'olmadığı', 'olmak', 'olması', 'olmayan', 'olmaz', 'olsa', 'olsun', 'olup', 'olur', 'olur',
    'olursa', 'oluyor', 'on', 'ön', 'ona', 'önce', 'ondan', 'onlar', 'onlara', 'onlardan', 'onları', 'onların', 'onu', 'onun',
    'orada', 'öte', 'ötürü', 'otuz', 'öyle', 'oysa', 'pek', 'rağmen', 'sana', 'sanki', 'sanki', 'şayet', 'şekilde', 'sekiz',
    'seksen', 'sen', 'senden', 'seni', 'senin', 'şey', 'şeyden', 'şeye', 'şeyi', 'şeyler', 'şimdi', 'siz', 'siz', 'sizden',
    'sizden', 'size', 'sizi', 'sizi', 'sizin', 'sizin', 'sonra', 'şöyle', 'şu', 'şuna', 'şunları', 'şunu', 'ta', 'tabii',
    'tam', 'tamam', 'tamamen', 'tarafından', 'trilyon', 'tüm', 'tümü', 'u', 'ü', 'üç', 'un', 'ün', 'üzere', 'var', 'vardı',
    've', 'veya', 'ya', 'yani', 'yapacak', 'yapılan', 'yapılması', 'yapıyor', 'yapmak', 'yaptı', 'yaptığı', 'yaptığını',
    'yaptıkları', 'ye', 'yedi', 'yerine', 'yetmiş', 'yi', 'yı', 'yine', 'yirmi', 'yoksa', 'yu', 'yüz', 'zaten', 'zira'
] # https://github.com/ahmetax/trstop/blob/master/dosyalar/turkce-stop-words

nltk_trstop = [
    'acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha',
    'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim',
    'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu',
    'tüm', 've', 'veya', 'ya', 'yani'
] # https://github.com/xiamx/node-nltk-stopwords/blob/master/data/stopwords/turkish

add_stop = [
    'a', 'b', 'c', 'ç', 'd', 'e', 'f', 'g', 'ğ', 'h', 'ı', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'ö', 'p', 'r', 's', 'ş', 't',
    'u', 'ü', 'v', 'y', 'z', 'li', 'lı', 'si', 'sı', 'te', 'ta', 'ın', 'in', 'na', 'ne', 'ler', 'lar', 'de', 'da', 'nın', 'nin',
    'lık', 'ım', 'im', 'yok', 'di', 'dı'
]

english_stop_words = stopwords.words('english')

stop_words = sorted(list(set(trstop).union(nltk_trstop).union(add_stop).union(english_stop_words)))

In [12]:
def monthly_tweets(year, month):
    
    tweets = []
    
    for tweet_list in df["tweets"]:
        for tweet in tweet_list:
            if tweet["twt_date"][:4] == str(year) + str(month):
                tweets.append(tweet["ref_twt_txt"])
    
    # Remove URLs
    tweets = [tweet[:re.search("http", tweet).start()] for tweet in tweets if "http" in tweet]
    # Convert to pandas Series
    tweets = pd.Series(tweets, name="tweets")
    # Lowercase Turkish letters
    tweets = tweets.apply(lambda x: x.replace("Ç", "ç").replace("Ğ", "ğ").replace("I", "ı").replace("İ", "i").replace("Ö", "ö").replace("Ş", "ş").replace("Ü", "ü"))
    # Lowercase all letters
    tweets = tweets.apply(lambda x: x.lower())
    # Replace newline characters ("\n") with space
    tweets = tweets.apply(lambda x: x.replace("\n", " "))
    # Replace non-letter characters with a space
    tweets = tweets.apply(lambda x: re.sub('[^A-Za-z#@çğıöşü]+',' ', x))
    # Stop word removal
    tweets = tweets.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    return tweets

In [13]:
def get_most_common_words_and_hashtags(year, month):
    
    tweets = monthly_tweets(str(year), str(month))
    
    words = []
    hashtags = []
    
    for tweet in tweets:
        for word in tweet.strip().split():
            if len(word) > 1:
                words.append(word)
            
            if word[0] == "#" and len(word) > 1:
                hashtags.append(word)
                
    return pd.concat([pd.Series([item[0] for item in Counter(words).most_common(200)], name=f"words_{year}{month}"), pd.Series([item[0] for item in Counter(hashtags).most_common(200)], name=f"hashtags_{year}{month}")], axis=1)

In [14]:
years = ["21", "22"]
months = ["01", "02", "03", "04", "05", "06", "07" ,"08", "09", "10", "11", "12"]

In [15]:
trends_df = pd.DataFrame()

for year in years:
    if year == "22":
        for month in months[:5]:
            print(f"Year: 20{year}, Month: {month} ...", end=" ")
            trends_df = pd.concat([trends_df, get_most_common_words_and_hashtags(year, month)], axis=1)
            print(f"completed!")
    else:
        for month in months:
            print(f"Year: 20{year}, Month: {month} ...", end=" ")
            trends_df = pd.concat([trends_df, get_most_common_words_and_hashtags(year, month)], axis=1)
            print(f"completed!")

Year: 2021, Month: 01 ... completed!
Year: 2021, Month: 02 ... completed!
Year: 2021, Month: 03 ... completed!
Year: 2021, Month: 04 ... completed!
Year: 2021, Month: 05 ... completed!
Year: 2021, Month: 06 ... completed!
Year: 2021, Month: 07 ... completed!
Year: 2021, Month: 08 ... completed!
Year: 2021, Month: 09 ... completed!
Year: 2021, Month: 10 ... completed!
Year: 2021, Month: 11 ... completed!
Year: 2021, Month: 12 ... completed!
Year: 2022, Month: 01 ... completed!
Year: 2022, Month: 02 ... completed!
Year: 2022, Month: 03 ... completed!
Year: 2022, Month: 04 ... completed!
Year: 2022, Month: 05 ... completed!


In [16]:
trends_df.head(60)

,words_2101,hashtags_2101,words_2102,hashtags_2102,words_2103,hashtags_2103,words_2104,hashtags_2104,words_2105,hashtags_2105,...,words_2201,hashtags_2201,words_2202,hashtags_2202,words_2203,hashtags_2203,words_2204,hashtags_2204,words_2205,hashtags_2205
0,mesut,#armandasaklı,büyük,#fbvgs,mart,#armandasaklı,nisan,#armandasaklı,beşiktaş,#şampiyonsensin,...,iyi,#ahmetçalık,güzel,#samsun,güzel,#uel,nisan,#ramazan,mayıs,#inadıylaşampiyon
1,özil,#fenersignmesut,fenerbahçe,#galatasaray,büyük,#bizimçocuklar,bugün,#yeniprofilresmi,mayıs,#aksadabaskınvar,...,güzel,#yeniprofilresmi,iyi,#yeniprofilresmi,mart,#gsvbjk,iyi,#boğazınkartalı,fenerbahçe,#ramazanbayramı
2,fenerbahçe,#bjkvgs,iyi,#turkishstudentslivesmatters,allah,#öğrencilerölmekistemiyor,güzel,#bitcoin,allah,#armandasaklı,...,ilk,#bitcoin,bugün,#survivor,iyi,#survivor,güzel,#yeniprofilresmi,iyi,#bjkvfb
3,yeni,#yeniprofilresmi,allah,#venividivici,türkiye,#kadinasiddetehayir,allah,#survivor,#şampiyonsensin,#mescidiaksa,...,allah,#z,allah,#ukrania,allah,#yeniprofilresmi,bugün,#fbvgs,güzel,#survivor
4,büyük,#survivor,ilk,#armandasaklı,iyi,#yeniprofilresmi,türkiye,#ramazan,israil,#gsvbjk,...,son,#boğazınkartalı,türkiye,#boğazınkartalı,bugün,#gsvbarça,allah,#survivor,türk,#annelergünü
5,ilk,#hayirlicumalar,yeni,#turkishstudentslivesmatter,güzel,#bjkvfb,devam,#liderbeşiktaş,şampiyon,#aksadabaskinvar,...,yeni,#survivor,ukrayna,#ukrayna,türkiye,#galatasaray,ilk,#survivorallstar,trabzonspor,#trabzonspor
6,bugün,#jummamubarak,devam,#yafenerya,bugün,#noticeturkishstudents,iyi,#galatasaray,güzel,#bjkvhty,...,bugün,#btc,ilk,#ukraina,galatasaray,#pakvaus,türk,#galatasaray,türkiye,#yeniprofilresmi
7,iyi,#gönüldağı,şubat,#erdoğanınyanındayız,devam,#galatasaray,ilk,#aydınlanmayangün,ilk,#survivor,...,devam,#nft,şubat,#ukrainerussia,ilk,#barçavgs,büyük,#kadirgecesi,bugün,#boğazınkartalı
8,güzel,#bjkvriz,bugün,#ertelemedeğiliptalistiyoruz,yeni,#beşiktaş,yeni,#beşiktaşınmaçıvar,türkiye,#genocideingaza,...,büyük,#sezenaksuhaddinibil,devam,#kiew,amp,#boğazınkartalı,türkiye,#şamp,kutlu,#efsane
9,devam,#galatasaray,türkiye,#whosaveturkishstudents,türk,#istanbulsözleşmesi,türk,#tamkapanma,iyi,#freepalestine,...,türkiye,#survivorallstar,büyük,#ukraine,büyük,#gönüldağı,son,#bjkvaly,büyük,#sessizistila


In [17]:
trends_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   words_2101     200 non-null    object
 1   hashtags_2101  200 non-null    object
 2   words_2102     200 non-null    object
 3   hashtags_2102  200 non-null    object
 4   words_2103     200 non-null    object
 5   hashtags_2103  200 non-null    object
 6   words_2104     200 non-null    object
 7   hashtags_2104  200 non-null    object
 8   words_2105     200 non-null    object
 9   hashtags_2105  200 non-null    object
 10  words_2106     200 non-null    object
 11  hashtags_2106  200 non-null    object
 12  words_2107     200 non-null    object
 13  hashtags_2107  200 non-null    object
 14  words_2108     200 non-null    object
 15  hashtags_2108  200 non-null    object
 16  words_2109     200 non-null    object
 17  hashtags_2109  200 non-null    object
 18  words_2110     200 non-null   

In [18]:
trends_df.to_excel("twitter_trends.xlsx", index=False)